In [2]:
import networkx as nx
import copy
import gurobipy as gp
from gurobipy import GRB
from helpers import *
import time
from ipynb.fs.full.MatrixModelFormulations import create_model, greedy_scheduling

# Instantiate processor

proc = Processor(4, 2, "HotspotConfiguration", 318.15)

# proc_cores is the number of processors per layer.
proc_layers = proc.nr_layers
proc_cores = proc.proc_per_layer

proc_total = proc.nr_pes
proc_config_path = proc.path_to_config


timestep = 1e-1

T_amb = proc.T_amb

coefficients = proc.coefficients


In [3]:
def create_RA_BC(graph : nx.DiGraph, restriction: list[list[int]], block: list[list[int]], MIPGap = False):
    """
    Solves the Restricted Assignment with Blocking Constraints problem (RA-BC).

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.
    - restriction : list[list[int]]
        A list of lists where each inner list contains the processor indices that a specific task can be scheduled on.
    - block : list[list[int]]
        A list of lists where each inner list contains the processor indices that are blocked for a specific task.
    - MIPGap : bool, optional
      If True, return the MIP gap along with other outputs. Default is False.

    Returns:
    --------
    - makespan : int
      The total time required to complete all tasks.
    - tasks_per_proc : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    - MIP gap : bool,optional
      The MIP gap, only returned if MIPGap is True.
    """
    nr_tasks = len(graph.nodes)
    nr_processors = proc_total
    task_lengths = [graph.nodes[i]['weight'] for i in graph.nodes]
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]
    
    # Create a new model
    m = gp.Model("Matching")

    # Create variables
    makespan = m.addVar(name="makespan")
    start_time = [m.addVar(name=f"s({i})") for i in range(nr_tasks)]
    x = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name = f"x[{i},{p}]") for p in range(nr_processors)] for i in range(nr_tasks)] # x[i,p] == 1 iff task i scheduled on PE p
    b = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"b[{i},{j}]") for j in range(nr_tasks)] for i in range(nr_tasks)] # b[i,j] == 1 iff task i finishes before task j starts
    o = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"o[{i},{j}]") for j in range(nr_tasks)] for i in range(nr_tasks)] # o[i,j] == 1 iff task i overlaps with task j

    # Set objective: minimize makespan
    m.setObjective(1.0 * makespan, GRB.MINIMIZE)

    # Define makespan
    m.addConstrs((start_time[i] + task_lengths[i] <= makespan for i in range(nr_tasks)), name="ms_def")

    # Task is planned on exactly one processor:
    m.addConstrs((sum(x[i][p] for p in range(nr_processors)) == 1 ) for i in range(nr_tasks))

    # Define b[i,j] and o[i,j]
    M = sum(task_lengths)  # Large constant: Worst case execution time, i.e. executing all tasks on the slowest core.
    m.addConstrs((start_time[i] + task_lengths[i] <= start_time[j] + M * (1 - b[i][j]) + M * o[i][j]) for i in range(nr_tasks) for j in range(nr_tasks))
    m.addConstrs((start_time[i] + M * (b[i][j] + o[i][j]) >= start_time[j] + task_lengths[j]) for i in range(nr_tasks) for j in range(nr_tasks))

    # If  two task on same processor, then one of them comes before the other
    m.addConstrs((x[i][p] + x[j][p] + o[i][j] <= 2) for i in range(nr_tasks) for j in range(nr_tasks) for p in range(nr_processors) if i != j)

    # Restrictions must be adhered to:
    m.addConstrs((sum(x[i][p] for p in restriction[i]) == 1 ) for i in range(nr_tasks))

    # Blocking tasks do not overlap
    m.addConstrs((x[u][p] + o[v][u] <= 1) for u in range(nr_tasks) for v in range(nr_tasks) for p in block[v])
    m.setParam('MIPGap', 0.25)
    m.setParam('TimeLimit', 1200)
    # Optimize model
    m.optimize()



    tasks_per_proc = [[] for _ in range(nr_processors)]

    # Print the values of all variables
    for i in range(nr_tasks):
        for p in range(nr_processors):
            if x[i][p].X == 1.0:
                print("Task", i, "is mapped on PE", p, ". start time:", start_time[i].X, ", finish time:", start_time[i].X + task_lengths[i])
                tasks_per_proc[p].append((i, start_time[i].X, start_time[i].X + task_lengths[i], task_power[i]))
    
    ms = round(makespan.X)
    gap = float(m.MIPGap)
    m.dispose()
    
    
    if MIPGap:
        return ms, tasks_per_proc, gap
    return ms, tasks_per_proc



In [4]:
def greedy_RA_BC(graph : nx.DiGraph, restriction: list[list[int]], block: list[list[int]]):
    """
    Approximates the Restricted Assignment with Blocking Constraints problem (RA-BC) greedily.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.
    - restriction : list[list[int]]
        A list of lists where each inner list contains the processor indices that a specific task can be scheduled on.
    - block : list[list[int]]
        A list of lists where each inner list contains the processor indices that are blocked for a specific task.

    Returns:
    --------
    - makespan : int
      The total time required to complete all tasks.
    - tasks_per_proc : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    """
    tpp = [[] for _ in range(proc_total)]
    nr_tasks = len(graph.nodes)
    task_lengths = [graph.nodes[i]['weight'] for i in graph.nodes]
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]
    
    makespan = 0

    def power_consumption_of_core(time, core):
        for t, st, ft, pow in tpp[core]:
            if round(st) <= time < round(ft):
                return pow
        return 0
    
    def is_blocked(time, core):
        for t, st, ft, pow in tpp[core]:
            if round(st) <= time < round(ft) and pow == 0:
                return True
        return False
    
    def is_idle(time, core):
        return power_consumption_of_core(time, core) == 0 and not(is_blocked(time, core))


    for i in range(nr_tasks):
        opt_t = None
        opt_p = None
        for p in restriction[i]:

            possible_time_stamps = []
            for t in range(makespan):
                if all([is_idle(time, p) for time in range(t, t + task_lengths[i])]) and all([power_consumption_of_core(time, proc) == 0 for time in range(t, t + task_lengths[i]) for proc in block[i]]):
                    possible_time_stamps.append(t)
            for t in possible_time_stamps:
                for t_run in range(t, t + task_lengths[i]):
                    if t_run not in possible_time_stamps and t_run <= makespan:
                        break
                else:
                    if opt_t == None or t < opt_t:
                        opt_t = t
                        opt_p = p
            else:
                if opt_t == None:
                    opt_t = makespan
                    opt_p = p
        
        tpp[opt_p].append((i, opt_t, opt_t + task_lengths[i], task_power[i]))
        for p in block[i]:
            tpp[p].append((i, opt_t, opt_t + task_lengths[i], 0))
        makespan = max(makespan, opt_t + task_lengths[i])
    
    return makespan, tpp

In [5]:
def restriction_blocking_set(stack: list[int], T_crit: float, T_amb: float):
    """
    Computes the power limits for tasks running on different subsets of PEs while considering temperature constraints.

    Parameters:
    -----------
    - stack : list[int]
      A list of integers representing the PE indices in a subset.
    - T_crit : float
      The critical temperature threshold for processors.
    - T_amb : float
      The ambiend temperature.

    Returns:
    --------
    list[tuple[float, list[int], list[int]]]: A list of tuples, where each tuple contains:
        - float: The maximum power output for the current subset of the stack.
        - list[int]: The current subset of the stack.
        - list[int]: The remaining elements of the original stack that are not in the current subset.
    """ 
    S_org = [s for s in stack]
    PowerLimits = []
    while len(stack) != 0:
        m = gp.Model("LP")
        x = m.addVar(lb=0, name="MaxPower")
        m.setObjective(x, GRB.MAXIMIZE)
        m.addConstrs((sum(coefficients[i][j] * x for j in stack) <= (T_crit - T_amb)) for i in stack)
        m.optimize()

        max_power = x.X
        PowerLimits.append((max_power, stack.copy(), [p for p in S_org if p not in stack]))
        stack.remove(min(stack))
    return PowerLimits


In [9]:
def MatrixModel_to_RA_BC_no_hor(graph: nx.DiGraph, T_crit: float, T_amb: float):
    """
    Computes the restriction and blocking sets for a Matrix Model instance, assuming no horizontal heat dissapation.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.
    - T_crit : float
      The critical temperature threshold for processors.
    - T_amb : float
      The ambiend temperature.

    Returns:
    --------
    - list[list[int]]
        A list of lists where each inner list contains the processor indices that a specific task can be scheduled on.
    - list[list[int]]
        A list of lists where each inner list contains the processor indices that are blocked for a specific task.

    """
    nr_tasks = len(graph.nodes)
    S = [[] for _ in range(proc_cores)]
    for i in range(proc_cores):
        S[i] = [i + proc_cores * x for x in range(proc_layers)]
    res = []
    block = []
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]

    PowerLimits = [restriction_blocking_set(s, T_crit, T_amb) for s in S]
    
    for task in range(nr_tasks):
        restriction = set()
        blocking = set()
        found = False
        for PL in PowerLimits:
            for (x, R, B) in PL:
                if task_power[task] <= x:
                    restriction = restriction.union(R)
                    blocking = blocking.union(B)
                    found = True
                    break
        if found == False:
            raise Exception("Unsolvable instance!")
        res.append(list(restriction))
        block.append(list(blocking))
    
    return res, block


In [10]:
def MatrixModel_to_RA_BC(graph : nx.DiGraph, T_crit: float, T_amb: float):
    """
    Computes the restriction and blocking sets for a Matrix Model instance, assuming there is horizontal heat dissapation.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.
    - T_crit : float
      The critical temperature threshold for processors.
    - T_amb : float
      The ambiend temperature.

    Returns:
    --------
    - list[list[int]]
        A list of lists where each inner list contains the processor indices that a specific task can be scheduled on.
    - list[list[int]]
        A list of lists where each inner list contains the processor indices that are blocked for a specific task.

    """
    P = list(range(proc_total))
    P_org = list(range(proc_total))
    PowerLimits = []
    nr_tasks = len(graph.nodes)
    res = []
    block = []
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]
    while len(P) != 0:
        m = gp.Model("LP")
        x = m.addVar(lb=0, name="MaxPower")
        m.setObjective(x, GRB.MAXIMIZE)
        m.addConstrs((sum(coefficients[i][j] * x for j in P) <= (T_crit - T_amb)) for i in P)
        m.optimize()
        max_power = x.X
        PowerLimits.append((max_power, P.copy(), [p for p in P_org if p not in P]))
        P.remove(max(P, key=lambda x: coefficients[x][x]))
    
    for task in range(nr_tasks):
        for (x, R, B) in PowerLimits:
            if task_power[task] <= x:
                res.append(R)
                block.append(B)
                break
        else:
            raise Exception("Instance unsolvable!")

    return res, block


In [ ]:
output_file_mm_normal = "output_matrixmodel_model.txt"
output_file_mm_greedy = "output_greedymatrixmodel_model.txt"

output_file_rabc_nohor_normal = "output_rabc_nohor_model_approx.txt"
output_file_rabc_nohor_greedy = "output_greedyrabc_nohor_model.txt"

output_file_rabc_normal = "output_file_rabc_normal_approx.txt"
output_file_rabc_greedy = "output_file_rabc_greedy.txt"


ms1, tpp1, ms2, tpp2 = None,None,None,None

found_no_hor = False
found = False

# Run Experiments
for i in range(1, 501, 1):
    # Solve MILP for Matrix Model
    task_graph = create_random_graph(i)
    start = time.time()
    ms, tpp = create_model(task_graph, 400)
    proc.tpp_to_power_trace(tpp, ms, output_file_mm_normal, 1e-1)
    t_max = proc.compute_max_ptrace_temp(output_file_mm_normal)
    f = open(output_file_mm_normal, 'a')
    f.write(f"Nr tasks {i}, Time {time.time() - start}, Makespan {ms}, Max Temp {t_max} \n")
    f.close()

    # Approximate  Matrix Model Greedily
    start = time.time()
    ms, tpp = greedy_scheduling(task_graph, 400)
    proc.tpp_to_power_trace(tpp, ms, output_file_mm_greedy, 1e-1)
    end = time.time()
    t_max, top1p = proc.compute_max_ptrace_temp(output_file_mm_greedy, True)
    f = open(output_file_mm_greedy, 'a')
    f.write(f"Nr tasks {i}, Time {end - start}, Makespan {ms}, Max Temp {t_max}, Top 1 percent highs {top1p} \n")
    f.close()

    # Solve MILP RA-BC formulation ignoring horizontal heat dissapation
    r, b = MatrixModel_to_RA_BC_no_hor(task_graph, 400, 318.15)

    start = time.time()
    ms, tpp, gap = create_RA_BC(task_graph, r, b, True)
    proc.tpp_to_power_trace(tpp, ms, output_file_rabc_nohor_normal, 1e-1)
    end = time.time()
    t_max, top1p = proc.compute_max_ptrace_temp(output_file_rabc_nohor_normal, True)
    f = open(output_file_rabc_nohor_normal, 'a')
    f.write(f"Nr tasks {i}, Time {end - start}, Makespan {ms}, Max Temp {t_max}, Top 1 percent highs {top1p}, GAP {gap * 100}% \n")
    f.close()

    # Approximate RA-BC formulation ignoring horizontal heat dissapation Greedily
    start = time.time()
    ms, tpp = greedy_RA_BC(task_graph, r, b)
    proc.tpp_to_power_trace(tpp, ms, output_file_rabc_nohor_greedy, 1e-1)
    end = time.time()
    t_max, top1p = proc.compute_max_ptrace_temp(output_file_rabc_nohor_greedy, True)
    f = open(output_file_rabc_nohor_greedy, 'a')
    f.write(f"Nr tasks {i}, Time {end - start}, Makespan {ms}, Max Temp {t_max}, Top 1 percent highs {top1p} \n")
    f.close()


    # Solve MILP RA-BC formulation
    r, b = MatrixModel_to_RA_BC(task_graph, 400, 318.15)
        
    start = time.time()
    ms, tpp, gap = create_RA_BC(task_graph, r, b, True)
    proc.tpp_to_power_trace(tpp, ms, output_file_rabc_normal, 1e-1)
    end = time.time()
    t_max, top1p = proc.compute_max_ptrace_temp(output_file_rabc_normal, True)
    f = open(output_file_rabc_normal, 'a')
    f.write(f"Nr tasks {i}, Time {end - start}, Makespan {ms}, Max Temp {t_max}, Top 1 percent highs {top1p}, GAP {gap * 100}% \n")
    f.close()

    # Approximate RA-BC formulation Greedily
    start = time.time()
    ms, tpp = greedy_RA_BC(task_graph, r, b)
    proc.tpp_to_power_trace(tpp, ms, output_file_rabc_greedy, 1e-1)
    end = time.time()
    t_max, top1p = proc.compute_max_ptrace_temp(output_file_rabc_greedy, True)
    f = open(output_file_rabc_greedy, 'a')
    f.write(f"Nr tasks {i}, Time {end - start}, Makespan {ms}, Max Temp {t_max}, Top 1 percent highs {top1p} \n")
    f.close()
